In [2]:
from openai import OpenAI
import csv
import random
import re
import os
from dotenv import load_dotenv
import pandas as pd
import time
import json

load_dotenv("key.env")
api_key = os.getenv("OPENAI_TOKEN")
client = OpenAI(api_key=api_key)
model_imit = "gpt-4.1"
model_eval = "gpt-4.1"

#decides if the LLM gets only a fragment of the novel or the complete 
imitate_style_from = "fragment" #"fragment" or "novel" 

In [3]:
#function to extract a 4000-words fragment starting from a new paragraph

def extract_fragment_from_txt(file_txt, number_of_words=4000):
    with open(file_txt, 'r', encoding='utf-8') as f:
        text = f.read()

    #non-empty paragraphs
    paragraphs = [p.strip() for p in text.split('\n') if p.strip()]

    max_attempts = 100
    for _ in range(max_attempts):
        idx_start = random.randint(0, len(paragraphs) - 1)
        first_paragraph = paragraphs[idx_start]

        #a paragraph should start with a capital letter (in some cases the .txt, as it is generated from a pdf, has 'paragraphs' that start with an incomplete sentence because it was on a new page)
        first_char_match = re.search(r'\S', first_paragraph)  #first char that is not a space
        if not first_char_match:
            continue  #empty paragraph

        first_char = first_char_match.group()
        if first_char.isalpha() and first_char.islower():
            continue  #not really a new paragraph but a new page

        #build the fragment adding paragraphs until 4000 words
        fragment = []
        total_words = 0

        for i in range(idx_start, len(paragraphs)):
            paragraph = paragraphs[i]
            words = re.findall(r'\b\w+\b', paragraph)
            total_words += len(words)
            fragment.append(paragraph)
            if total_words >= number_of_words:
                return '\n\n'.join(fragment)

    raise ValueError("A fragment of the desired number of words (or more) could not be extracted.")

In [4]:
file = "SanManuelBuenoMartir.txt"
#"delfrioalfuego.txt"
#"los4jinetesdelapocalipsis.txt" 
#"SanManuelBuenoMartir.txt"
#"tristana_textoplano.txt"

book = file.split('.')[0] #for the name of the excel file

fragment = extract_fragment_from_txt(file, number_of_words=4000)
print(f"Length in words: {len(re.findall(r'\b\w+\b', fragment))}")
print(fragment)  

Length in words: 4133
Trabajaba también manualmente, ayudando con sus brazos a ciertas labores del pueblo. En la temporada de trilla íbase a la era a trillar y aventar, y en tanto aleccionaba o distraía a los labradores, a quienes ayudaba en estas faenas. Sustituía a las veces a algún enfermo en su ta- rea. Un día del más crudo invierno se encontró con un niño, muertito de frío, a quien su padre le enviaba a recoger una res a larga distancia, en el monte.

-Mira -le dijo al niño-, vuélvete a casa a

calentarte, y dile a tu padre que yo voy a hacer el encargo.

Y al volver con la res se encontró con el padre, todo confuso, que iba a su encuentro. En invierno partía leña para los pobres. Cuan- do se secó aquel magnífico nogal -"un nogal matriarcal" le llamaba-, a cuya sombra había jugado de niño y con cuyas nueces se había du- rante tantos años regalado, pidió el tronco, se lo llevó a su casa y, después de labrar en él seis tablas, que guardaba al pie de su lecho, hizo del resto leña par

In [5]:
#LLM GENERATES A FRAGMENT THAT IMITATES THE STYLE OF THE ORIGINAL ONE (either from the whole novel or one fragment)

def generate_fragment_imitation(prompt_intro, novel_text, model_imit, client):
    """
    Generates a literary fragment imitating the style of a given text (fragment or full novel),
    and returns the generated fragment along with its word count.
    Parameters:
        prompt_intro (str): Introductory instruction for the model (in Spanish).
        novel_text (str): Text whose style the model has to imitate (either whole novel or a fragment).
        model_imit (str): Name of the model to use for generation.
        client: Configured OpenAI client instance.
    Returns:
        extracted_fragment (str), word_count (int)
    """

    # Construct the full prompt to send to the model
    full_prompt = f"{prompt_intro}\n\n{novel_text}"

    # Generate a continuation in the same style using the model
    response = client.chat.completions.create(
        model=model_imit,
        messages=[
            {"role": "system", "content": "Eres un escritor que imita estilos literarios con precisión."},
            {"role": "user", "content": full_prompt}
        ],
    )

    generated_text = response.choices[0].message.content

    # Extract the generated fragment between << and >> and count words
    match = re.search(r'<<(.+?)>>', generated_text, re.DOTALL)
    if match:
        extracted_fragment = match.group(1).strip()
        words = re.findall(r'\b\w+\b', extracted_fragment)
        word_count = len(words)
    else:
        extracted_fragment = ""
        word_count = 0

    return extracted_fragment, word_count


In [6]:
#CREATE ASSISTANT FOR EVALUATION (only once)
# Check if assistant ID is saved
assistant_file = "assistant_id.json"

if os.path.exists(assistant_file):
    # Load existing assistant ID
    with open(assistant_file, "r") as f:
        assistant_id = json.load(f)["assistant_id"]
else: 
    # Create assistant only once
    assistant = client.beta.assistants.create(
        name="Evaluador de estilos de novelas",
        instructions="Eres un crítico literario especializado en analizar el estilo narrativo de novelas. Tu tarea es examinar textos y ofrecer una evaluación detallada del estilo, incluyendo tono, estructura, uso del lenguaje, ritmo y voz narrativa.",
        model=model_eval
    )
    assistant_id = assistant.id
    # Save to file
    with open(assistant_file, "w") as f:
        json.dump({"assistant_id": assistant_id}, f)

In [ ]:
MAX_ITERATIONS = 4
iteration = 1
result_eval = "NO"

registro_iteraciones = []

initial_prompt = "A continuación tienes un texto de una novela. Tienes que escribir un texto en el mismo estilo de unas 4000 palabras. Delimita el fragmento utilizando << y >>."

if imitate_style_from == "fragment":
    novel_text = extract_fragment_from_txt(file, number_of_words=4000)
else:
    with open(file, 'r', encoding='utf-8') as f:
        novel_text = f.read()

while iteration <= MAX_ITERATIONS:
    print(f"Iteración {iteration}")

    #1) Imitación del estilo

    extracted_fragment, word_count = generate_fragment_imitation(initial_prompt, novel_text, model_imit, client)

    #2) Crear hilo
    thread = client.beta.threads.create()

    #3) Crear mensaje para evaluar estilo
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=(
            "A continuación tienes dos textos, etiquetados como TEXTO 1 y TEXTO 2. Quiero que evalúes si tienen el mismo estilo y podrían haber sido escritos por la misma persona. "
            "Razona en detalle tu respuesta y devuelve el resultado final en formato: <<RESULTADO FINAL: SÍ>> o <<RESULTADO FINAL: NO>>.\n\n"
            f"TEXTO 1:\n{fragment}\n\n"
            f"TEXTO 2:\n{extracted_fragment}"
        )
    )

    #4) Ejecutar evaluación
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            break
        time.sleep(1)

    #5) Obtener respuesta
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    assistant_reply = messages.data[0].content[0].text.value  

    #6) Evaluar resultado
    if "RESULTADO FINAL: SÍ" in assistant_reply:
        result_eval = "SÍ"
    else:
        result_eval = "NO"

    iteracion_info = {
        "iteracion": iteration,
        "prompt": initial_prompt,
        "fragmento_original": novel_text,
        "fragmento_generado": extracted_fragment,
        "num_palabras": word_count,
        "evaluacion": assistant_reply.strip(),
        "resultado_final": result_eval
    }

    #7) Si es SÍ, terminar
    if result_eval == "SÍ":
        print("Resultado final del análisis: SÍ")
        registro_iteraciones.append(iteracion_info)
        break

    #8) Si es NO, pedir un nuevo prompt y volver a intentar
    print("Resultado final del análisis: NO. Solicitando nuevo prompt...")

    solicitud_mejora = (
        f"El TEXTO 2 es una imitación del estilo del TEXTO 1 generada por un LLM, a partir del prompt "
        f"\"{initial_prompt}\" "
        "Teniendo en cuenta los problemas que has detectado en el TEXTO 2, proporciona un prompt mejorado, dándole indicaciones adicionales sobre el estilo que debe adoptar."
        "Usa { y } para indicar el principio y el final del nuevo prompt."
    )

    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user", 
        content=solicitud_mejora
    )

    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            break
        time.sleep(1)

    messages = client.beta.threads.messages.list(thread_id=thread.id)
    nueva_respuesta = messages.data[0].content[0].text.value

    #Extraer el nuevo prompt desde la respuesta
    start_idx = nueva_respuesta.find("{") + 1
    end_idx = nueva_respuesta.find("}")
    nuevo_prompt_generado = nueva_respuesta[start_idx:end_idx].strip()

    #Añadir a la info de iteración
    iteracion_info["solicitud_mejora_prompt"] = solicitud_mejora
    iteracion_info["nuevo_prompt_generado"] = nuevo_prompt_generado

    registro_iteraciones.append(iteracion_info)

    #Usar el nuevo prompt para la siguiente iteración
    initial_prompt = nuevo_prompt_generado
    iteration += 1

#Guardar en fichero 
output_file = "registro_iteraciones.json"

#Cargar contenido previo si existe
if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        datos_anteriores = json.load(f)
else:
    datos_anteriores = []

#Combinar datos previos con los nuevos
datos_actualizados = datos_anteriores + registro_iteraciones

#Guardar el archivo actualizado
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(datos_actualizados, f, ensure_ascii=False, indent=2)
if result_eval == "NO":
    print("No se logró imitar el estilo tras 4 intentos.")


Iteración 1
Resultado final del análisis: NO. Solicitando nuevo prompt...
Iteración 2
Resultado final del análisis: NO. Solicitando nuevo prompt...
Iteración 3
Resultado final del análisis: SÍ
